In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
from tkinter.font import Font

class Line:
    def __init__(self, file_name):
        self.file_name = file_name
        self.line = []
        self.ip_add = []

    def read(self):
        with open(self.file_name, "r") as f:
            self.line = f.readlines()
        self.ip_address()

    def ip_address(self):
        self.ip_add = list(map(lambda x: x.split(" ")[0], self.line))
        return self.ip_add

    def less_than(self, z):
        self.less = list(map(lambda x: x.split(".")[0], self.ip_add))
        l = []
        for i in self.less:
            if i.isdigit():
                if int(i) < z:
                    l.append(int(i))
        return l

    def ratio(self, z=5):
        total_ips = len(self.ip_address())
        less_than_list = self.less_than(z)
        if len(less_than_list) == 0:
            return 0
        return total_ips / len(less_than_list)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("🌸 Light Blue IP Analyzer 🌸")
        self.root.geometry("600x600")
        self.root.configure(bg="#add8e6")  # Light blue background

        # Custom fonts
        self.header_font = Font(family="Papyrus", size=24, weight="bold")
        self.button_font = Font(family="Comic Sans MS", size=14, weight="bold")
        self.label_font = ("Helvetica", 14)
        self.result_font = ("Helvetica", 16, "italic")

        # Style for ttk widgets
        style = ttk.Style()
        style.configure('TButton', font=self.button_font)
        style.configure('TEntry', font=self.button_font)

        # Header Label
        header = ttk.Label(self.root, text="🌸 IP Address Analyzer 🌸", font=self.header_font, background="#add8e6", foreground="#d146a8")
        header.pack(pady=15)

        # Frame for buttons
        btn_frame = ttk.Frame(self.root)
        btn_frame.pack(pady=10)

        self.btn_select = ttk.Button(btn_frame, text="📁 Select Log File", command=self.load_file)
        self.btn_select.grid(row=0, column=0, padx=15, pady=8)

        self.btn_show_ips = ttk.Button(btn_frame, text="📝 Show IP List", command=self.show_ips_popup, state="disabled")
        self.btn_show_ips.grid(row=0, column=1, padx=15, pady=8)

        # Frame for ratio input and button
        ratio_frame = ttk.Frame(self.root)
        ratio_frame.pack(pady=20)

        # Label for threshold
        self.threshold_label = ttk.Label(ratio_frame, text="Threshold (z):", font=self.label_font, background="#add8e6", foreground="black")
        self.threshold_label.grid(row=0, column=0, padx=8, pady=5)

        # Entry styled similar to button (but plain)
        self.entry_z = ttk.Entry(ratio_frame, width=6)
        self.entry_z.insert(0, "5")
        self.entry_z.grid(row=0, column=1, padx=8, pady=5)

        # Calculate button
        self.btn_ratio = ttk.Button(ratio_frame, text="📊 Calculate Ratio", command=self.calculate_ratio, state="disabled")
        self.btn_ratio.grid(row=0, column=2, padx=15, pady=5)

        # Output label (no box, cleaner look)
        self.lbl_result = ttk.Label(self.root, text="", font=self.result_font, background="#add8e6", foreground="#8b008b")
        self.lbl_result.pack(pady=15)

        # Initialize variables
        self.file_path = ""
        self.line_obj = None

    def load_file(self):
        file_path = filedialog.askopenfilename(title="Select Log File", filetypes=[("Text files", "*.txt *.log"), ("All files", "*.*")])
        if file_path:
            self.file_path = file_path
            self.line_obj = Line(self.file_path)
            self.line_obj.read()
            self.btn_show_ips.config(state="normal")
            self.btn_ratio.config(state="normal")
            self.lbl_result.config(text="")
            messagebox.showinfo("File Loaded", f"Successfully loaded:\n{self.file_path}")

    def show_ips_popup(self):
        if self.line_obj:
            ips = self.line_obj.ip_address()
            # Create popup window
            popup = tk.Toplevel()
            popup.title("🌸 List of IP Addresses 🌸")
            popup.geometry("470x400")
            popup.configure(bg="#add8e6")

            lbl = ttk.Label(popup, text="IP Addresses:", font=("Helvetica", 16, "bold"), background="#add8e6", foreground="#d146a8")
            lbl.pack(pady=10)

            frame = ttk.Frame(popup)
            frame.pack(fill='both', expand=True, padx=10, pady=5)

            scrollbar = ttk.Scrollbar(frame)
            scrollbar.pack(side='right', fill='y')

            listbox = tk.Listbox(frame, yscrollcommand=scrollbar.set, font=("Courier New", 10), bg="#fff0f5")
            for ip in ips:
                listbox.insert(tk.END, ip)
            listbox.pack(side='left', fill='both', expand=True)

            scrollbar.config(command=listbox.yview)

    def calculate_ratio(self):
        z_str = self.entry_z.get()
        if z_str.isdigit():
            z = int(z_str)
            ratio_value = self.line_obj.ratio(z)
            self.lbl_result.config(text=f"Total IPs / IPs with first octet < {z}:\n{ratio_value:.2f}")
        else:
            messagebox.showerror("Invalid input", "Please enter a valid integer for z.")

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()